In [1]:
%matplotlib inline

----------------------------
#Session 0 
----------------------------

-------------------------
# Tensors

Tensors are a specialized data structure that are very similar to arrays and matrices.
In PyTorch, we use tensors to encode the inputs and outputs of a model, as well as the model’s parameters.

Tensors are similar to `NumPy’s <https://numpy.org/>`_ ndarrays, except that tensors can run on GPUs or other hardware accelerators. In fact, tensors and
NumPy arrays can often share the same underlying memory, eliminating the need to copy data (see `bridge-to-np-label`). Tensors
are also optimized for automatic differentiation (we'll see more about that later in the `Autograd <autogradqs_tutorial.html>`__
section). If you’re familiar with ndarrays, you’ll be right at home with the Tensor API. If not, follow along!


In [2]:
import torch
import numpy as np

## Initializing a Tensor

Tensors can be initialized in various ways. Take a look at the following examples:

**Directly from data**

Tensors can be created directly from data. The data type is automatically inferred.



In [3]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

**From a NumPy array**

Tensors can be created from NumPy arrays (and vice versa - see `bridge-to-np-label`).



In [4]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

**From another tensor:**

The new tensor retains the properties (shape, datatype) of the argument tensor, unless explicitly overridden.



In [5]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.3913, 0.2999],
        [0.4212, 0.3940]]) 



**With random or constant values:**

``shape`` is a tuple of tensor dimensions. In the functions below, it determines the dimensionality of the output tensor.



In [6]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

# You can also try zeros_like, rand_like and ones_like
another_rand_tensor = torch.rand_like(zeros_tensor)
another_ones_tensor = torch.ones_like(zeros_tensor)
another_zeros_tensor = torch.zeros_like(zeros_tensor)

print(f"Other Random Tensor: \n {another_rand_tensor} \n")
print(f"Other Ones Tensor: \n {another_ones_tensor} \n")
print(f"Other Zeros Tensor: \n {another_zeros_tensor}")

Random Tensor: 
 tensor([[0.6645, 0.5553, 0.9879],
        [0.7283, 0.7297, 0.7592]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])
Other Random Tensor: 
 tensor([[0.8183, 0.0079, 0.1042],
        [0.2355, 0.7635, 0.5322]]) 

Other Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Other Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


--------------




## Attributes of a Tensor

Tensor attributes describe their shape, datatype, and the device on which they are stored.



In [7]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


--------------




## Operations on Tensors

Over 100 tensor operations, including arithmetic, linear algebra, matrix manipulation (transposing,
indexing, slicing), sampling and more are
comprehensively described `here <https://pytorch.org/docs/stable/torch.html>`__.

Each of these operations can be run on the GPU (at typically higher speeds than on a
CPU). If you’re using Colab, allocate a GPU by going to Runtime > Change runtime type > GPU.

By default, tensors are created on the CPU. We need to explicitly move tensors to the GPU using
``.to`` method (after checking for GPU availability). Keep in mind that copying large tensors
across devices can be expensive in terms of time and memory!



In [8]:
# We move our tensor to the GPU if available
if torch.cuda.is_available():
    tensor = tensor.to('cuda')
    print(f"Device tensor is stored on: {tensor.device}")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

tensor = tensor.to(device) #same as tensor.to('cuda') or tensor.to('cpu')

Try out some of the operations from the list.
If you're familiar with the NumPy API, you'll find the Tensor API a breeze to use.




**Standard numpy-like indexing and slicing:**



In [9]:
tensor = torch.ones(3, 4)
print('First row: ', tensor[0])
print('First column: ', tensor[:, 0])
print('Last column:', tensor[..., -1])
tensor[:,1] = 0
print(tensor)

First row:  tensor([1., 1., 1., 1.])
First column:  tensor([1., 1., 1.])
Last column: tensor([1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


**Joining tensors** You can use ``torch.cat`` to concatenate a sequence of tensors along a given dimension.
See also `torch.stack <https://pytorch.org/docs/stable/generated/torch.stack.html>`__,
another tensor joining op that is subtly different from ``torch.cat``.



In [10]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


**Arithmetic operations**



In [11]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.empty_like(y1)
torch.matmul(tensor, tensor.T, out=y3)

print(f'Equivalent computation: {torch.all(y1==y2) and torch.all(y2==y3)}')


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.empty_like(tensor)
torch.mul(tensor, tensor, out=z3)
print(f'Equivalent computation: {torch.all(z1==z2) and torch.all(z2==z3)}')

Equivalent computation: True
Equivalent computation: True


**in-place vs out-of-place operations**



In [12]:
# methods ending with a _ are in place
x = torch.zeros(2,3)
print(f'x at the begining:\n {x}')
x.add(1)
print(f'x after out-of-place op:\n {x}')
x.add_(1)
print(f'x after in-place op:\n {x}')

# when such in-place variant does not exist, one may rely on the `out` parameter
torch.add(x,1, out=x)
print(f'x after another in-place op:\n {x}')

x at the begining:
 tensor([[0., 0., 0.],
        [0., 0., 0.]])
x after out-of-place op:
 tensor([[0., 0., 0.],
        [0., 0., 0.]])
x after in-place op:
 tensor([[1., 1., 1.],
        [1., 1., 1.]])
x after another in-place op:
 tensor([[2., 2., 2.],
        [2., 2., 2.]])


**Single-element tensors** If you have a one-element tensor, for example by aggregating all
values of a tensor into one value, you can convert it to a Python
numerical value using ``item()``:



In [13]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

9.0 <class 'float'>


-------------------------
# Exercises on Tensor manipulation

## Exercise 1: 
Compute $\Vert t_1-t_2\Vert^2$ and return the result as a `float`. 
You should use `torch.square` and `torch.sum` in your solution.

<u>Hint:</u> Beware, that $t_1$ and $t_2$ have different shapes, so you should resize them to a common shape first.

In [14]:
debug = False
def exercise1(t1,t2):
  return 0

def test_exercise1():
  t1 = torch.ones((3,4))
  t2 = torch.arange(0,12)
  result = exercise1(t1,t2)
  expected = 386
  if isinstance(result, float) and result == expected:
    print('Exercise 1: OK')
  else:
    print('Exercise 1: NOK')
    if not isinstance(result, float):
      print('Hint: don\'t forget to return a float and not a tensor (with item())')
    if debug:
      print(f' result : {result}')
      print(f' expected : {expected}')

test_exercise1()

Exercise 1: NOK
Hint: don't forget to return a float and not a tensor (with item())


## Exercice 2: Monte-carlo estimation of $\pi/4$
To estimate $\pi/4$, we can generate $n$ random samples in the unit square, and count the ratio that have a norm less than 1. To do so in pytorch, generate a random tensor of shape $n\times 2$ and compute the norm of each column, and guess the rest of the solution.
![Illustration](https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Pi_30K.gif/440px-Pi_30K.gif)


In [15]:
debug = False
def exercise2(n_samples):
  result = torch.tensor(0).double()
  return result

def test_exercise2():
  result = exercise2(int(1e7))
  expected = torch.tensor([np.pi/4]).double()
  if  torch.isclose(result, expected, atol=1e-3):
    print('Exercise 2: OK')
  else:
    print('Exercise 2: NOK')
    if debug:
      print(f' result : {result.item()}')
      print(f' expected : {expected.item()}')

test_exercise2()

Exercise 2: NOK


## Exercise 3: In-place computation of the Fibonacci number
Denoting $A=\begin{pmatrix} 1& 1\\ 1 &0\end{pmatrix}$, one may show that $A^n=\begin{pmatrix} F_{n+1}& F_n \\ F_n & F_{n-1}\end{pmatrix}$. Using the previous fact as well as $\frac{F_{n+1}}{F_n}\to \frac{1+\sqrt{5}}{2}$, you shall compute $\phi:=\frac{1+\sqrt{5}}{2}$ based only on inline operations. In practice, you should initialize $A$ to the correct matrix, and compute iteratively $A^{2n}$ in-place (using the `out` parameter of the `torch.mm` routine).

In [16]:
debug = False
import math

def exercise3(A, n):
  '''Here you should compute A^{2n} for the Fibonnaci matrix. The computation must be in-place'''
  pass

def test_exercise3():
  A = torch.empty((2,2))
  exercise3(A,3)
  result = A[0,0]/A[1,0]
  expected = (1+torch.sqrt(torch.tensor(5.)))/2
  if  torch.isclose(result, expected, 1e-3):
    print('Exercise 3: OK')
  else:
    print('Exercise 3: NOK')
    if debug:
      print(f' result : {result}')
      print(f' expected : {expected}')

test_exercise3()

Exercise 3: NOK
